# Test notebook for visualizing surfaces in 3D, Neurohackweek 2017

- Cloned from the test_3d.ipynb notebook
- Initial functionality based on [IPyvolume](https://ipyvolume.readthedocs.io/en/latest/index.html)
- modified by Melanie to focus on surface viewer
- added function showSurface.py to show surfaces
- furthe rmodified by Murat and Melanie to allow for interactiveness

autoreload the functions/packages that are imported

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

load required packages

In [3]:
%matplotlib inline
import os
import sys
import matplotlib.pyplot as plt

import nibabel as nb
import ipyvolume
import ipyvolume.pylab as p3
import numpy as np
import niwidgets
from ipywidgets import interact,fixed,IntSlider

%run showSurface.py

load data

In [4]:
# before running, set the global variable NHW to your NHW_visualization git directory
# e.g. add 'export NHW=/Users/ely/Desktop/nhw2017/NHW_visualization' to your ~/.bashrc file

nhw = os.environ['NHW'] # set the global variable NHW to your NHW_visualization git directory
V = nb.load(nhw + '/data/100610/MNINonLinear/T1w_restore_brain.nii.gz').get_data()

In [5]:
data_lh = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.L.midthickness_MSMAll.32k_fs_LR.surf.gii')
data_lh_myelin = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.L.MyelinMap.32k_fs_LR.func.gii')

data_rh = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.R.midthickness_MSMAll.32k_fs_LR.surf.gii')
data_rh_myelin = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.R.MyelinMap.32k_fs_LR.func.gii')

## Plot gifti surface and overlay files for a subject's left and right hemisphere

In [16]:
showSurface(data_lh,data_lh_myelin)

A Jupyter Widget

In [17]:
showSurface(data_rh,data_rh_myelin,newfigure=False)

## now read in a FreeSurfer surface and curvature/annotation as overlay

In [8]:
showSurface(nhw+'/data/006/surf/lh.inflated',nhw+'/data/006/surf/lh.curv')

A Jupyter Widget

In [9]:
showSurface(nhw+'/data/006/surf/lh.inflated',nhw+'/data/006/surf/lh.aparc.annot')

A Jupyter Widget

now let's choose a specific frame of a list of overlay files

In [10]:
showSurface(nhw+'/data/006/surf/lh.inflated',[nhw+'/data/006/surf/lh.thickness',nhw+'/data/006/surf/lh.curv'],1)

A Jupyter Widget

# Now we are making this (manually) interactive 
this will later be added as niwidgets_surface to niwidgets

In [12]:
interact(showSurface,surface=fixed(data_lh),overlay = fixed(data_lh_myelin),newfigure=fixed(True),
         colormap=['summer','autumn','winter','spring'],figsize=fixed(np.array([600,600])),
                figlims=fixed(np.array([[-100,100],[-100,100],[-100,100]])))

A Jupyter Widget

<function __main__.showSurface>

In [13]:
overlays = [None,nhw+'/data/006/surf/lh.thickness',nhw+'/data/006/surf/lh.curv']

interact(showSurface,surface=fixed(nhw+'/data/006/surf/lh.inflated'),
         overlay = fixed(overlays),
         frame = IntSlider(value=0,description='frame',min=0,max=len(overlays)-1),
         newfigure=fixed(True),
         colormap=['summer','autumn','winter','spring'],figsize=fixed(np.array([600,600])),
         figlims=fixed(np.array([[-100,100],[-100,100],[-100,100]])))

A Jupyter Widget

<function __main__.showSurface>

since this update sthe whole figure and hence flashed, we tried to be smart and write a wrapper inside niwidgets_surface, which unfortunately still has a bug